In [1]:
import requests 
import pandas as pd
import json
import pickle

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from keras.models import load_model
import numpy as np

Using TensorFlow backend.


In [3]:
train = pd.read_csv('train.csv')
train.dropna(inplace = True)
X = train['TEXT']
y = train['truth']
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.1, shuffle = False)

vectorizer = TfidfVectorizer()
vectors_train = vectorizer.fit_transform(xtrain)

rfc = pickle.load(open('RandomForest.sav', 'rb'))
gbc = pickle.load(open('Gradient-Boost.sav', 'rb'))
etc = pickle.load(open('Extra-Trees.sav', 'rb'))
adb = pickle.load(open('AdaBoost.sav', 'rb'))
bnb = pickle.load(open('BernoulliNB.sav', 'rb'))
cnn = load_model('CNN.h5')

In [4]:
teamToken = "s00AbOoUxTwwSP2hag4bgKe+q0g93uDhidc04pAXuas="
get = 'https://erisk.irlab.org/challenge-service/getwritings/'+teamToken
post= 'https://erisk.irlab.org/challenge-service/submit/'+teamToken+'/'

r = requests.get(url = get)
data = r.json()

In [5]:
main = []

nick = "nick"
dec = "decision"
sc = "score"
count = 1

r = requests.get(url = get)
data = r.json()
for d in data:
    main.append([d['content'], d['nick']])
maindf = pd.DataFrame(main, columns = ['content', 'nick'])

print('Got Sequence ', count)
    
while(len(data) != 0):
    tempdf = pd.DataFrame()
    vectors = vectorizer.transform(maindf['content'])    
    for i in range(5):
        send = []
        if(i == 0):
            print('Processing Run 1')
            decision = bnb.predict(vectors)
            prob = bnb.predict_proba(vectors)
            score = [max(p) for p in prob]
        elif(i == 1):    
            print('Processing Run 2')
            decision = rfc.predict(vectors)
            prob = bnb.predict_proba(vectors)
            score = [max(p) for p in prob]
        elif(i == 2):  
            print('Processing Run 3')
            decision = gbc.predict(vectors)
            prob = bnb.predict_proba(vectors)
            score = [max(p) for p in prob]
        elif(i == 3):    
            print('Processing Run 4')
            decision = etc.predict(vectors)
            prob = bnb.predict_proba(vectors)
            score = [max(p) for p in prob]
        else:    
            print('Processing Run 5')
            decision = adb.predict(vectors)
            prob = bnb.predict_proba(vectors)
            score = [max(p) for p in prob]

        for k in range(len(maindf)):
                send.append({nick:str(maindf['nick'][k]),
                              dec:int(decision[k]),
                              sc:float(score[k])})
        
        requests.post(url = post+str(i), json = send)
        
    r = requests.get(url = get)
    
    count += 1
    print('Got Sequence ', count)
    
    data = r.json()
    
    templ = []
    for d in data:
           templ.append([d['content'], d['nick']])
    tempdf = pd.DataFrame(templ, columns = ['content', 'nick'])
       
    for i1, r1 in maindf.iterrows():
        for i2, r2 in tempdf.iterrows():
            if(r1['nick'] == r2['nick']):
                 r1['content'] = r1['content'] + r2['content']           

Got Sequence  1
Processing Run 1
Processing Run 2
Processing Run 3
Processing Run 4
Processing Run 5
Got Sequence  2
Processing Run 1
Processing Run 2
Processing Run 3
Processing Run 4
Processing Run 5
Got Sequence  3
Processing Run 1
Processing Run 2
Processing Run 3
Processing Run 4
Processing Run 5
Got Sequence  4
Processing Run 1
Processing Run 2
Processing Run 3
Processing Run 4
Processing Run 5
Got Sequence  5
Processing Run 1
Processing Run 2
Processing Run 3
Processing Run 4
Processing Run 5
Got Sequence  6
Processing Run 1
Processing Run 2
Processing Run 3
Processing Run 4
Processing Run 5
Got Sequence  7
Processing Run 1
Processing Run 2
Processing Run 3
Processing Run 4
Processing Run 5
Got Sequence  8
Processing Run 1
Processing Run 2
Processing Run 3
Processing Run 4
Processing Run 5
Got Sequence  9
Processing Run 1
Processing Run 2
Processing Run 3
Processing Run 4
Processing Run 5
Got Sequence  10
Processing Run 1
Processing Run 2
Processing Run 3
Processing Run 4
Proces

KeyboardInterrupt: 